# EnergyPlus Parameterization and Sensitivity Analysis for Residential Single Family Homes


## Daniel Xu, Keller Lab

This notebook was created in order to document the workflow and processes of EnergyPlus within Python. This is for my project involving the parameterization of EnergyPlus. Given that EnergyPlus has over 2000 available inputs, more research needs to be done in order to document which parameters are most significant and relevant in home energy modeling and simulations. 

The goal of this project is to run EnergyPlus on a model single family residential home. Some of the inputs will be fixed to default variables. Other variables, such as window thickness, thermostat set-points, etc. will be varied on pre-defined distributions. 

The distributions will be defined as seen in this paper: 
https://www.sciencedirect.com/science/article/abs/pii/S037877881631372X?via=ihub

### Importing necessary libraries

In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

### Changing the working directory 

In [2]:
os.chdir("/jumbo/keller-lab/projects/eplus_sensitivity/scripts")
print("Current working directory:", os.getcwd())

Current working directory: /jumbo/keller-lab/projects/eplus_sensitivity/scripts


### Setting EnergyPlus Input Data Dictionary

In [3]:
iddfile = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/Energy+.idd"
IDF.setiddname(iddfile)

### Setting a Skeleton IDF File 

This modifiable IDF file is provided by EnergyPlus in its initial download. It provides a mock residential home in Chicago with multiple zones. I will assume that some of the inputs are fixed and others are varied on a distribution. 

Directly from the EnergyPlus documentation: 

"This file does the basic test of an air distribution system in a residential home. A two speed heat pump with a supplmental gas heater provides space heating and cooling. It provides ventilation through the ZoneAirBalance:OutdoorAir model."

In [4]:
idfname = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/ExampleFiles/SingleFamilyHouse_TwoSpeed_MultiStageElectricSuppCoil.idf"

preidf = IDF(idfname)

# preidf.printidf()

print(preidf.idfobjects['Site:Location'])

[
Site:Location,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longitude
    -6,                       !- Time Zone
    190;                      !- Elevation
]


In [12]:
import requests
from datetime import datetime, timedelta

def get_active_weather_stations(api_key, lat, lon):
    base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/stations'
    headers = {'token': api_key}
    # Define the date one year ago from today
    one_year_ago = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    # Set the request to retrieve more stations for a broader check
    params = {
        'extent': f'{lat-0.05},{lon-0.05},{lat+0.05},{lon+0.05}',
        'limit': '1000',  # Adjust limit as needed
        'sortfield': 'mindate',
        'sortorder': 'desc'
    }

    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return []

    try:
        data = response.json()
        active_stations = []
        # Filter stations by checking if the 'maxdate' is within the last year
        for station in data.get('results', []):
            if station['maxdate'] >= one_year_ago:
                active_stations.append(station)
        return active_stations
    except ValueError:
        print("Failed to decode JSON from response.")
        return []

api_key = 'SrgpVmvZhbtZXRSdBgknhaRSQlhTNzBt'
latitude = 41.78  
longitude = -87.75  

active_stations = get_active_weather_stations(api_key, latitude, longitude)
print(f"Found {len(active_stations)} active stations:")
for station in active_stations:
    print(station)


Found 4 active stations:
{'elevation': 191.1, 'mindate': '2009-04-01', 'maxdate': '2024-05-07', 'latitude': 41.730399, 'name': 'OAK LAWN 1.1 N, IL US', 'datacoverage': 0.9999, 'id': 'GHCND:US1ILCK0139', 'elevationUnit': 'METERS', 'longitude': -87.75615}
{'elevation': 185.8, 'mindate': '1997-05-01', 'maxdate': '2024-05-09', 'latitude': 41.78412, 'name': 'CHICAGO MIDWAY AIRPORT, IL US', 'datacoverage': 1, 'id': 'GHCND:USW00014819', 'elevationUnit': 'METERS', 'longitude': -87.75514}
{'elevation': 185.8, 'mindate': '1948-01-01', 'maxdate': '2024-05-11', 'latitude': 41.78412, 'name': 'CHICAGO MIDWAY AIRPORT, IL US', 'datacoverage': 1, 'id': 'WBAN:14819', 'elevationUnit': 'METERS', 'longitude': -87.75514}
{'elevation': 189, 'mindate': '1928-02-01', 'maxdate': '2024-05-09', 'latitude': 41.73727, 'name': 'CHICAGO MIDWAY AIRPORT 3 SW, IL US', 'datacoverage': 0.9993, 'id': 'GHCND:USC00111577', 'elevationUnit': 'METERS', 'longitude': -87.77734}


Based on this location, we will then use the diyepw package to create time series weather data. More information about diyepw can be found at: 

https://diyepw.readthedocs.io/en/latest/tutorial.html (PNNL)

In [6]:
import diyepw

diyepw.create_amy_epw_files_for_years_and_wmos(
    years=[year1, year2],  # replace with actual years of interest
    wmos=[wmo_index],  # replace with the WMO index you found
    max_records_to_interpolate=10,
    max_records_to_impute=25,
    max_missing_amy_rows=5,
    allow_downloads=True
)
